In [4]:
#Have to restart notebook after installing this
!python3 -m pip install kfp==1.8.14 --upgrade --user

In [9]:
!python3 -m pip install scikit-learn

In [12]:
import kfp
import kfp.dsl as dsl
from kfp.v2.dsl import component, Input, Output, InputPath, OutputPath, Dataset, Metrics, Model, Artifact

In [14]:
@component(
    packages_to_install = ["pandas", "scikit-learn"],
)
def load(data: Output[Dataset]):
    import pandas as pd
    from sklearn import datasets

    dataset = datasets.load_iris()
    df = pd.DataFrame(data=dataset.data, columns= ["Petal Length", "Petal Width", "Sepal Length", "Sepal Width"])
    
    df.to_csv(data.path)

@component(
    packages_to_install = ["pandas"],
)
def print_head(data: Input[Dataset]):
    import pandas as pd
    df = pd.read_csv(data.path)
    print(df.head())

@dsl.pipeline(
    name='Iris',
    description='iris'
)
def pipeline():
    load_task = load()
    print_task = print_head(data=load_task.outputs["data"])

kfp.compiler.Compiler(mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE).compile(
    pipeline_func=pipeline,
    package_path='iris_csv.yaml')